In [171]:
import plotly
import cufflinks
import plotly.offline as py
import plotly.graph_objs as go
import plotly.express as px
import pandas as pd
import numpy as np
import cvxopt as opt
from cvxopt import blas, solvers
import yfinance as yf

np.random.seed(123)

In [172]:
prices = yf.download(["PETR4.SA", 'BBDC4.SA', 'TASA4.SA', 'PRIO3.SA'])['Adj Close']

[*********************100%***********************]  4 of 4 completed


In [173]:
returns = np.log(prices).diff().fillna(0)
solvers.options['show_progress'] = False

In [174]:
## NUMBER OF ASSETS
n_assets =returns.shape[1]

## NUMBER OF OBSERVATIONS
n_obs = returns.shape[0]

return_vec = np.random.randn(n_assets, n_obs)

In [175]:
def rand_weights(n):
    k = np.random.rand(n)
    return k / sum(k)

In [176]:
def random_portfolio(returns):

    risk_free_rate = 0.000358
    
    p = np.asmatrix(np.mean(returns, axis=1))
    w = np.asmatrix(rand_weights(returns.shape[0]))
    C = np.asmatrix(np.cov(returns))
    
    mu = (w * p.T)

    sigma = np.sqrt(w * C * w.T)
    sharpe = (mu - risk_free_rate) / sigma 
    
    return mu, sigma, sharpe

In [188]:
n_portfolios = 1000
means, stds, sharpe = np.column_stack([
    random_portfolio(return_vec) 
    for _ in range(n_portfolios)
])

In [189]:
def optimal_portfolio(returns):
    n = len(returns)
    returns = np.asmatrix(returns)
    
    N = 100
    mus = [10**(5.0 * t/N - 1.0) for t in range(N)]
    
    # Convert to cvxopt matrices
    S = opt.matrix(np.cov(returns))
    pbar = opt.matrix(np.mean(returns, axis=1))
    
    # Create constraint matrices
    G = -opt.matrix(np.eye(n))   # negative n x n identity matrix
    h = opt.matrix(0.0, (n ,1))
    A = opt.matrix(1.0, (1, n))
    b = opt.matrix(1.0)
    
    # Calculate efficient frontier weights using quadratic programming
    portfolios = [solvers.qp(mu*S, -pbar, G, h, A, b)['x'] 
                  for mu in mus]
    ## CALCULATE RISKS AND RETURNS FOR FRONTIER
    returns = [blas.dot(pbar, x) for x in portfolios]
    risks = [np.sqrt(blas.dot(x, S*x))  for x in portfolios]
    ## CALCULATE THE 2ND DEGREE POLYNOMIAL OF THE FRONTIER CURVE
    m1 = np.polyfit(returns, risks, 2)
    x1 = np.sqrt(m1[2] / m1[0])
    # CALCULATE THE OPTIMAL PORTFOLIO
    wt = solvers.qp(opt.matrix(x1 * S), -pbar, G, h, A, b)['x']
    
    return np.asarray(wt), returns, risks

In [190]:
weights, returns, risks = optimal_portfolio(return_vec)

In [191]:
weights

array([[4.13046891e-01],
       [4.18034486e-01],
       [1.68918535e-01],
       [8.83008444e-08]])

In [192]:
data = [go.Scatter(
            x=stds.flatten(),
            y=means.flatten(),
            mode='markers',            
             marker=dict(
                    color=sharpe.flatten(),
                    showscale=False,
                    cmax=sharpe.flatten().max(),
                    cmin=0,                 
                    )       
        ),
        go.Scatter(
            x=risks,
            y=returns,
            mode='lines+markers',            
        ),    
        go.Scatter(
            x=stds[sharpe.argmax()],
            y=means[sharpe.argmax()],
            mode='markers',            
        ),                    
       ]

title_text = "Risco x Retorno para os portfólios gerados"
title={'text': title_text, 'xanchor': 'center', 'yanchor': 'bottom', 'y':0, 'x':0.5,}

fig = go.Figure(data=data)

fig.update_layout(title=title, 
            xaxis_rangeslider_visible=False,  width=1280, height=720,
            xaxis_showgrid=True, xaxis_gridwidth=1, xaxis_gridcolor='#E8E8E8',
            yaxis_showgrid=True, yaxis_gridwidth=1, yaxis_gridcolor='#E8E8E8',
            yaxis_title="Retorno",
            xaxis_title="Risco",                  
            plot_bgcolor='rgba(0,0,0,0)'              
                 )
                  

fig.show()